In [1]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config) 

from sentence_transformers import SentenceTransformer, util
from tensorflow.keras.layers import Layer,Input,Dense,Lambda
from tensorflow.keras import Model
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text
from glob import glob
import setting as set_fnc
import copy as cp

use_enc_large = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
use_enc_small = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:0a:00.0, compute capability: 7.0



# Load datasets

In [2]:
mode = 'test'
q_lan = ['ar','de','es','hi','vi','zh']
d_lan = ['en']
corpus = 'MLQA'

df_question = {}
df_paragraph = {}
df_doc = {}
for lan in q_lan:
    df_question.update({
        f'en-{lan}':pd.read_csv(f'../datasets/{corpus}/{mode}/{corpus.lower()}_question_en-{lan}.csv')
    }) 
    df_doc.update({
        f'en-{lan}':pd.read_csv(f'../datasets/{corpus}/{mode}/{corpus.lower()}_doc_en-{lan}.csv')
    })

# mUSE Example

In [3]:
mUSE_model = hub.load(f'../models/{corpus}/finetuned_USE_MLQA_train_en-ar_de_es_zh_top0-0_q-d-distillation_1000MSE_0.01MSEq_1.0MSEd_0.0001MSEqd_0.0005LR_teacher_best_teacher_batchsize_16_acc_metric_3term')
for lan in q_lan[:]:
    doc_context_id = df_doc[f'en-{lan}']['doc_id'].to_list()
    doc_context_encoded = mUSE_model(df_doc[f'en-{lan}']['doc'].to_list()).numpy()
    
    question_id = df_question[f'en-{lan}']['doc_id'].to_list()
    questions = mUSE_model(df_question[f'en-{lan}']['question'].to_list()).numpy()
    
    top_1,top_5,top_10,mrr = set_fnc.evaluate(question_id,questions,doc_context_id,doc_context_encoded)

    print(f'distil comp-{lan}')
    precision = top_1 / len(questions)
    print(f"Traninng Score P@1: {precision:.3f}")
    precision = top_5 / len(questions)
    print(f"Traninng Score P@5: {precision:.3f}")
    precision = top_10 / len(questions)
    print(f"Traninng Score P@10: {precision:.3f}")
    print(f"Mrr score:{mrr:.3f}")

100%|██████████| 517/517 [00:00<00:00, 2834.51it/s]


distil comp-ar
Traninng Score P@1: 0.485
Traninng Score P@5: 0.689
Traninng Score P@10: 0.764
Mrr score:0.570


  0%|          | 0/500 [00:00<?, ?it/s]

distil comp-de
Traninng Score P@1: 0.648
Traninng Score P@5: 0.824
Traninng Score P@10: 0.877
Mrr score:0.723


  0%|          | 0/507 [00:00<?, ?it/s]

...

distil comp-hi
Traninng Score P@1: 0.020
Traninng Score P@5: 0.047
Traninng Score P@10: 0.091
Mrr score:0.035


  0%|          | 0/504 [00:00<?, ?it/s]

distil comp-vi
Traninng Score P@1: 0.280
Traninng Score P@5: 0.364
Traninng Score P@10: 0.425
Mrr score:0.318


100%|██████████| 504/504 [00:00<00:00, 3191.87it/s]

distil comp-zh
Traninng Score P@1: 0.579
Traninng Score P@5: 0.778
Traninng Score P@10: 0.847
Mrr score:0.665


# Sentence-Transformer Example

In [ ]:
ST_model = SentenceTransformer('XXXXX')
for lan in q_lan[:]:
    set_fnc.sent_bert_encode(ST_model,'ST_model',lan,df_doc[f'en-{lan}'],df_question)
    print('*'*50)